In [42]:
import pandas as pd

In [43]:
data = pd.read_csv('covid_confirmed_usafacts.csv')

data = data.melt(id_vars=["countyFIPS","County Name", "State", "StateFIPS"], var_name="date", value_name="value")

data.head(4)


,countyFIPS,County Name,State,StateFIPS,date,value
0,0,Statewide Unallocated,AL,1,2020-01-22,0
1,1001,Autauga County,AL,1,2020-01-22,0
2,1003,Baldwin County,AL,1,2020-01-22,0
3,1005,Barbour County,AL,1,2020-01-22,0


In [44]:
data = data[['countyFIPS','value','date']]

data['date'] = pd.to_datetime(data['date'])

data = data.rename(columns={"countyFIPS": "fips", "value": "act_cases"})

confirmed_cases = data

data.head(10)


,fips,act_cases,date
0,0,0,2020-01-22
1,1001,0,2020-01-22
2,1003,0,2020-01-22
3,1005,0,2020-01-22
4,1007,0,2020-01-22
5,1009,0,2020-01-22
6,1011,0,2020-01-22
7,1013,0,2020-01-22
8,1015,0,2020-01-22
9,1017,0,2020-01-22


In [45]:
item = []
for i in range(6):
    item.append('fcast_data' + str(i+1) + '.csv')    

print(item)

data = pd.concat(map(pd.read_csv, item), ignore_index=True)

['fcast_data1.csv', 'fcast_data2.csv', 'fcast_data3.csv', 'fcast_data4.csv', 'fcast_data5.csv', 'fcast_data6.csv']


C:\Users\ChrisEvans\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
data = data[data.target == "1 wk ahead inc case"]
data = data[data.location_name != data.State]
data['date'] = pd.to_datetime(data['target_end_date'])

data = data.rename(columns={'point': "for_cases"})

data = data[['model','date','fips','for_cases']]

forcasted_cases = data

forcasted_cases.head(10)


,model,date,fips,for_cases
10600,Columbia,2020-08-08,2013,0
10601,Columbia,2020-08-08,2016,0
10602,Columbia,2020-08-08,2020,745
10603,Columbia,2020-08-08,2050,0
10604,Columbia,2020-08-08,2060,0
10605,Columbia,2020-08-08,2068,0
10606,Columbia,2020-08-08,2070,0
10607,Columbia,2020-08-08,2100,0
10608,Columbia,2020-08-08,2105,0
10609,Columbia,2020-08-08,2110,14


In [47]:
forcasted_cases['fips'] = forcasted_cases['fips'].astype(str)
forcasted_cases['date'] = forcasted_cases['date'].astype(str)

confirmed_cases['fips'] = confirmed_cases['fips'].astype(str)
confirmed_cases['date'] = confirmed_cases['date'].astype(str)

all_cases = pd.merge(forcasted_cases, confirmed_cases, on=['date','fips'], how='inner')

all_cases.head(10)

,model,date,fips,for_cases,act_cases
0,Columbia,2020-08-08,2013,0,2
1,Ensemble,2020-08-08,2013,0,2
2,Columbia,2020-08-08,2016,0,4
3,Ensemble,2020-08-08,2016,2,4
4,Columbia,2020-08-08,2020,745,2175
5,Ensemble,2020-08-08,2020,589,2175
6,LANL,2020-08-08,2020,589,2175
7,Columbia,2020-08-08,2050,0,22
8,Ensemble,2020-08-08,2050,2,22
9,LANL,2020-08-08,2050,2,22
